In [29]:
import pandas
import duckdb
from src.club_names import name_dict

In [2]:

players = pandas.read_csv(r"data/fifa/players_21.csv")
results = pandas.read_csv(r"data/results_premier/results.csv", encoding = "ISO-8859-1")



In [7]:
query = """
    SELECT 
      club_name
      , avg(overall) filter(where club_position = 'RES') as reserve_overall
      , avg(overall) filter(where club_position = 'SUB') as subs_overall
      , avg(overall) filter(where club_position not in ('RES','SUB') ) as titular_overall
      , avg(overall) filter(where club_position in ('ST','CAM','LS','LW', 'RW','CF','RS' , 'LM' , 'RF') ) as attack_overall
      , avg(overall) filter(where club_position in ( 'GK', 'LCB', 'RCB' ,'CDM', 'LDM' , 'RDM', 'LB', 'RB' ,'CB', 'RCB' , 'LCB' ) ) as defend_overall

    FROM players 
    where league_name = 'English Premier League' 
    group by club_name
    
"""
teams_21 = duckdb.query(query).to_df()


In [35]:

query = """
    SELECT 
        Season ,
        HomeTeam,  
        AwayTeam,
        FTHG as goals_home,
        FTAG as goals_away
        
    FROM results
    where 
        season = '2020-21'
"""
results_21 = duckdb.query(query).to_df()

In [38]:
results_21["away_code"] = results_21.AwayTeam.apply(lambda x: name_dict[x])
results_21["home_code"] = results_21.HomeTeam.apply(lambda x: name_dict[x])

In [45]:
query = """
    SELECT 
        home_code as home
        ,away_code as away
        ,goals_home
        ,goals_away
        , th.reserve_overall as home_reserve_overall
        , th.subs_overall as home_subs_overall
        , th.titular_overall as home_titular_overall
        , th.attack_overall as home_attack_overall
        , th.defend_overall as home_defend_overall
        , ta.reserve_overall as away_reserve_overall
        , ta.subs_overall as away_subs_overall
        , ta.titular_overall as away_titular_overall
        , ta.attack_overall as away_attack_overall
        , ta.defend_overall as away_defend_overall
    FROM results_21 r
    Left Join teams_21 th on r.home_code = th.club_name
    Left Join teams_21 ta on r.away_code = ta.club_name
"""
df = duckdb.query(query).to_df()

In [49]:
df.to_csv('data/ml/df.csv', index=False)